In [1]:
import requests, json, time, re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt

In [2]:
sub_all = '/top.json?t=all'
sub_gilded = '/gilded.json'
sub_hot = '.json'

In [3]:
sub_new = '/new.json'

In [4]:
# adapted code for Adi
# posts is the returned dictionary (should pass in previously ran `posts`)
# subreddit = url string 
# subpage = the page on the subreddit (should be already defined)
def sub_query(subreddit, subpage, posts=None, pages=40):
    
    #Step 0: set initial conditions
    if posts == None:
        posts = []
    else:
        posts = posts
    url = f'https://old.reddit.com/r/{subreddit}{subpage}'
    after = None

    #Step 1: build the loop and print URL
    for _ in range(pages):
        if after == None:
            current_url = url
        elif subpage == '/top.json?t=all':
            current_url = url + '&after=' + after
        else:
            current_url = url + '?count=25&after=' + after

        print(current_url)

        #Step 2a: make the requests and handle status code. Add time.sleep now (2.b)
        res = requests.get(current_url, headers={'User-agent': 'hehehehe'})

        if res.status_code != 200:
            print('Status error', res.status_code)
            break

        #Step 3: Actually deal with the data   
        current_dict = res.json()
        current_posts = [p['data'] for p in current_dict['data']['children']]
        posts.extend(current_posts)

        after = current_dict['data']['after']

        #Step 2b:
        time.sleep(0.75)

    return posts

In [5]:
eli5 = sub_query('explainlikeimfive', sub_all)

https://old.reddit.com/r/explainlikeimfive/top.json?t=all
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_6pfe0k
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_6m4neu
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_6de844
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_6c5e97
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_707jhj
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_67msip
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_5m9asi
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_7482ri
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_6tfs11
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_60mib5
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_9tbb6a
https://old.reddit.com/r/explainlikeimfive/top.json?t=all&after=t3_atcf5o
https://old.reddit.com/r/explainlikeimfive/top.json?t=

In [6]:
eli5 = sub_query('explainlikeimfive', sub_hot, eli5)

https://old.reddit.com/r/explainlikeimfive.json
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cbmw88
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cbozix
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cbmqro
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cbjizb
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cbftrk
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cb8mdj
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cb71xn
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cb2ycb
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cazdmv
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_catb9m
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cau0ud
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_caq8ue
https://old.reddit.com/r/explainlikeimfive.json?count=25&after=t3_cahdeu
htt

In [7]:
eli5 = sub_query('explainlikeimfive', sub_new, eli5)

https://old.reddit.com/r/explainlikeimfive/new.json
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cbo6qv
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cblvpv
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cbjfx8
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cbgn73
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cbcld7
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cb8syq
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cb5fq9
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cb1q2d
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_caxy08
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_cau0ud
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_caqsbi
https://old.reddit.com/r/explainlikeimfive/new.json?count=25&after=t3_camqyc
https://old.reddit.com/r

In [8]:
def combine_text(posts):
    return[' '.join([post['title'], post['selftext']]) for post in posts] # build this out to a function

In [9]:
nsq = sub_query('NoStupidQuestions', sub_all)

https://old.reddit.com/r/NoStupidQuestions/top.json?t=all
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_acjhwt
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_a7ameq
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_ae9guo
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_b8sgjd
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_atdter
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_amocyi
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_abthzm
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_c8m5dz
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_bsf9w0
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_a30ky9
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_b5ez15
https://old.reddit.com/r/NoStupidQuestions/top.json?t=all&after=t3_7sv9yn
https://old.reddit.com/r/NoStupidQuestions/top.json?t=

In [10]:
nsq = sub_query('NoStupidQuestions', sub_hot, nsq)

https://old.reddit.com/r/NoStupidQuestions.json
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbm0v9
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbm9ya
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbpdnt
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbozd0
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbojmg
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbmcjc
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cblw1d
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbn41h
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbphqg
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbl2l3
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbp48r
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cbowb9
https://old.reddit.com/r/NoStupidQuestions.json?count=25&after=t3_cboiw3
htt

In [11]:
nsq = sub_query('NoStupidQuestions', sub_new, nsq)

https://old.reddit.com/r/NoStupidQuestions/new.json
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbpggn
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbp4yq
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cboxyj
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbooi4
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbofbk
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbo36v
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbnqwa
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbnk9a
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbna7f
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbn34o
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbmogs
https://old.reddit.com/r/NoStupidQuestions/new.json?count=25&after=t3_cbmcmj
https://old.reddit.com/r

In [12]:
eli5_text = combine_text(eli5)

In [13]:
nsq_text = combine_text(nsq)

In [14]:
eli5_df = pd.DataFrame(eli5_text, columns=['text'])
eli5_df['is_eli5']= 1

In [15]:
nsq_df = pd.DataFrame(nsq_text, columns=['text'])
nsq_df['is_eli5']= 0

In [16]:
reddit = pd.concat([eli5_df, nsq_df], ignore_index=True)

In [17]:
reddit['is_eli5'].value_counts()

1    2953
0    2949
Name: is_eli5, dtype: int64

In [18]:
reddit.shape

(5902, 2)

In [21]:
reddit = reddit.drop_duplicates()

In [26]:
reddit.shape

(3938, 2)

In [23]:
reddit['is_eli5'].value_counts(normalize=True)

0    0.512189
1    0.487811
Name: is_eli5, dtype: float64

In [25]:
reddit.to_csv('./datasets/reddit.csv', index=False)